# **Non Extreme Distribution (NED)**

In [ ]:
"""
Non-Extreme Distribution Function

This function calculates the **non-extreme distribution (NED)** metric for a software system based on
its partitioning. The NED metric measures the proportion of classes assigned to partitions of
extreme sizes (very small or very large), helping to identify imbalances in the partitioning.

Parameters:
- partition_class_bcs_assignment (dict): A dictionary where keys represent partitions, and values
  are dictionaries containing the 'classes' assigned to each partition.
  Example:
  {
      'partition_1': {'classes': ['ClassA', 'ClassB']},
      'partition_2': {'classes': ['ClassC', 'ClassD']}
  }
- result (dict, optional): An intermediate result (defaults to None). This parameter allows flexibility
  for pre-processed data or alternative representations of partition assignments.

Returns:
- float: The non-extreme distribution (NED) value, which ranges from 0 to 1. A value closer to 0
  indicates extreme partitioning sizes, while a value closer to 1 represents balanced partitioning.

Steps:
1. Count the total number of classes and the number of classes in partitions of "non-extreme" size.
2. Calculate the NED metric as the proportion of classes in extreme-sized partitions.
3. Return the NED value rounded to three decimal places.
"""

def non_extreme_distribution(partition_class_bcs_assignment, result=None):
    """
    Calculate the non-extreme distribution (NED) metric for a partitioning of classes.

    Parameters:
    - partition_class_bcs_assignment (dict): Partition assignment of classes.
    - result (dict, optional): Intermediate result. Defaults to None.

    Returns:
    - float: Non-extreme distribution value.
    """
    # Step 1: Use the provided partition assignment or initialize `result` with the input
    if result is None:
        result = partition_class_bcs_assignment

    # Extract the cluster (partition) names and count the number of clusters
    clusters = list(result.keys())
    n_clusters = len(clusters)  # Total number of partitions (clusters)

    # Initialize variables for calculating the metric
    sum_non_extreme = 0  # Accumulate the size of partitions in the non-extreme range
    total_classes = 0    # Total number of classes across all partitions

    # Step 2: Iterate through each partition
    for cluster in clusters:
        # Number of classes in the current partition
        size = len(partition_class_bcs_assignment[cluster]['classes'])
        total_classes += size  # Add to the total class count

        # Check if the partition size is within the "non-extreme" range (5 <= size <= 20)
        if 5 <= size <= 20:
            sum_non_extreme += size  # Add to the non-extreme class count

    # Step 3: Calculate the non-extreme distribution (NED) metric
    # NED is initialized to 1 (ideal balance). Adjust based on partition sizes.
    ned = 1
    if total_classes > 0 and sum_non_extreme > 0:
        # Subtract the proportion of classes in non-extreme partitions from 1
        ned = ned - (sum_non_extreme / total_classes)

    # Step 4: Return the NED value rounded to three decimal places
    return round(ned, 3)

# **Conceptual Modularity Quality (CMQ)**

In [ ]:
from collections import defaultdict

def calculate_ccoh(partition_class_bcs_assignment, runtime_call_volume):
    """
    Compute **cohesion** (ccohi) within each partition.

    Cohesion measures the degree of relatedness among classes within a partition.
    Higher cohesion indicates that classes within a partition are strongly related.

    Parameters:
    - partition_class_bcs_assignment (dict): Mapping of partitions to their assigned classes.
      Example:
      {
          'PartitionA': {'classes': ['Class1', 'Class2']},
          'PartitionB': {'classes': ['Class3']}
      }
    - runtime_call_volume (set): Set of runtime call relationships (not used in this function but
      passed for consistency with other metrics).

    Returns:
    - list: A list of cohesion (ccoh) values, one for each partition.
      Example: [0.8, 0.9]
    """
    ccoh = []  # Initialize the list to store cohesion values

    # Iterate over each partition in the assignment
    for partition in partition_class_bcs_assignment.values():
        classes = partition['classes']  # Classes belonging to the current partition
        Ni = len(classes)  # Total number of classes in the partition

        # Count unique pairs of classes, excluding self-references
        ui = sum(1 for c1 in classes for c2 in classes if c1 != c2)

        # Calculate cohesion using the formula: ui / (Ni * (Ni - 1))
        # Avoid division by zero when Ni <= 1
        ccoh_i = ui / (Ni * (Ni - 1)) if Ni > 1 else 0

        # Append the cohesion value for the current partition
        ccoh.append(ccoh_i)

    return ccoh

def calculate_ccop(partition_class_bcs_assignment, runtime_call_volume):
    """
    Compute **coupling** (ccopi;j) between partitions.

    Coupling measures the degree of interaction between classes from different partitions.
    Lower coupling indicates better modularity, as partitions are more independent.

    Parameters:
    - partition_class_bcs_assignment (dict): Mapping of partitions to their assigned classes.
    - runtime_call_volume (set): Set of runtime call relationships.
      Each item is a tuple representing a call trace (source, target).
      Example: {('Class1', 'Class3'), ('Class2', 'Class4')}

    Returns:
    - list: A list of coupling (ccop) values, one for each pair of partitions.
      Example: [0.1, 0.2]
    """
    ccop = []  # Initialize the list to store coupling values
    partitions = list(partition_class_bcs_assignment.keys())  # List of partition names
    M = len(partitions)  # Total number of partitions

    # Iterate over all unique pairs of partitions (i, j)
    for i in range(M):
        for j in range(i + 1, M):
            # Retrieve classes belonging to partitions i and j
            classes_i = partition_class_bcs_assignment[partitions[i]]['classes']
            classes_j = partition_class_bcs_assignment[partitions[j]]['classes']
            Ni = len(classes_i)  # Number of classes in partition i
            Nj = len(classes_j)  # Number of classes in partition j

            # Count interactions (call traces) between classes in partitions i and j
            sigma_ij = sum(1 for trace in runtime_call_volume
                           if trace[0] in classes_i and trace[1] in classes_j)

            # Calculate coupling using the formula: sigma_ij / (2 * Ni * Nj)
            # Avoid division by zero when Ni * Nj <= 0
            ccop_ij = sigma_ij / (2 * Ni * Nj) if Ni * Nj > 0 else 0

            # Append the coupling value for the current partition pair
            ccop.append(ccop_ij)

    return ccop

def calculate_cmq(partition_class_bcs_assignment, runtime_call_volume):
    """
    Compute **Conceptual Modularity Quality** (CMQ).

    CMQ combines cohesion (ccoh) and coupling (ccop) to provide a single metric for modularity.
    Higher CMQ values indicate better modularity, with high cohesion and low coupling.

    Parameters:
    - partition_class_bcs_assignment (dict): Mapping of partitions to their assigned classes.
    - runtime_call_volume (set): Set of runtime call relationships.

    Returns:
    - float: The CMQ value.
      Example: 0.75
    """
    # Step 1: Calculate cohesion (ccoh) values for all partitions
    ccoh_values = calculate_ccoh(partition_class_bcs_assignment, runtime_call_volume)

    # Step 2: Calculate coupling (ccop) values for all partition pairs
    ccop_values = calculate_ccop(partition_class_bcs_assignment, runtime_call_volume)

    # Step 3: Compute the number of partitions
    M = len(ccoh_values)

    # Step 4: Calculate CMQ numerator (sum of cohesion values)
    numerator = sum(ccoh_values)

    # Step 5: Calculate the average coupling between partitions
    denominator = M * (M - 1) / 2  # Total number of unique partition pairs
    avg_ccop = sum(ccop_values) / denominator if denominator > 0 else 0  # Avoid division by zero

    # Step 6: Compute the CMQ value using the formula: (sum(ccoh) / M) - avg(ccop)
    CMQ = (numerator / M) - avg_ccop

    return CMQ


# Example usage of the CMQ function
# Ensure partition assignments and runtime call volume data are provided
# Example input data (user-defined based on system under analysis)
# CMQ_value = calculate_cmq(partition_class_bcs_assignment, runtime_call_volume)
# print("CMQ Value:", CMQ_value)

# **Precision/Recall**

In [ ]:

from typing import Dict, List, Set, Tuple, Union
from itertools import combinations


# ---------------------------
# Validation Approach
# To ensure a fair and context-sensitive evaluation of decomposition quality, we employed a hybrid approach combining two complementary precision-recall metrics. The first metric, pairwise-based precision and recall, is particularly suited for overly fragmented decompositions, where the number of predicted partitions significantly exceeds the ground truth. This metric evaluates whether class pairs that should co-exist in the same service are preserved, making it ideal for assessing the cohesion of predictions in high-fragmentation scenarios. In contrast, the second metric, microservice-level precision and recall, is more appropriate when the predicted partitioning is closer in scale to the ground truth. It evaluates how well each predicted cluster aligns with the most similar ground truth cluster, capturing service-level cohesion and coupling. By dynamically selecting the metric based on the ratio of predicted to actual service count, we align the evaluation with the structure of each decomposition, avoiding bias toward either under- or over-fragmentation. 
# ---------------------------

# ---------------------------
# Input Cluster Dictionaries
# ---------------------------

# Ground truth (flat format)
flat_ground_truth_dict = {}

# Predicted clusters (flat format)
partition_class_assignment = {}

# ---------------------------
# Helper Functions
# ---------------------------

ClusterDict = Dict[str, Union[List[str], Dict[str, List[str]]]]

def pairs_from_partitions(partitions: ClusterDict) -> Set[frozenset]:
    if all(isinstance(v, dict) and "classes" in v for v in partitions.values()):
        iter_lists = (v["classes"] for v in partitions.values())
    elif all(isinstance(v, list) for v in partitions.values()):
        iter_lists = partitions.values()
    else:
        raise ValueError("Mixed or unsupported partition format detected.")
    pair_set = set()
    for cls_list in iter_lists:
        clean = [c for c in set(cls_list) if c]
        for a, b in combinations(clean, 2):
            pair_set.add(frozenset((a, b)))
    return pair_set

def flatten_clusters(d: ClusterDict) -> List[Set[str]]:
    clusters: List[Set[str]] = []
    for value in d.values():
        if isinstance(value, list):
            if value:
                clusters.append(set(value))
        elif isinstance(value, dict):
            for classes in value.values():
                if classes:
                    clusters.append(set(classes))
    return clusters

def best_overlap(cluster: Set[str], gold_clusters: List[Set[str]]) -> Set[str]:
    best, best_score = set(), -1.0
    for gold in gold_clusters:
        score = len(cluster & gold) / len(cluster) if cluster else 0
        if score > best_score:
            best, best_score = gold, score
    return best

def precision_recall_microservice(pred_clusters: List[Set[str]], gold_clusters: List[Set[str]]) -> Tuple[float, float]:
    precisions, recalls = [], []
    for pred in pred_clusters:
        gold = best_overlap(pred, gold_clusters)
        inter = len(pred & gold)
        precisions.append(inter / len(pred) if pred else 0)
        recalls.append(inter / len(gold) if gold else 0)
    return (sum(precisions) / len(precisions),
            sum(recalls) / len(recalls))

# ---------------------------
# Hybrid Evaluation Logic
# ---------------------------

num_predicted = len(partition_class_assignment)
num_ground_truth = len(flat_ground_truth_dict)

if num_predicted > 2 * num_ground_truth:
    # Use Metric Formula 1 (Pairwise)
    predicted_pairs = pairs_from_partitions(partition_class_assignment)
    actual_pairs = pairs_from_partitions(flat_ground_truth_dict)
    tp_pairs = predicted_pairs & actual_pairs
    precision = len(tp_pairs) / len(predicted_pairs) if predicted_pairs else 0.0
    recall = len(tp_pairs) / len(actual_pairs) if actual_pairs else 0.0
    metric_used = "Precision/Recall"
else:
    # Use Metric Formula 2 (Microservice-level)
    pred_clusters = flatten_clusters(partition_class_assignment)
    gold_clusters = flatten_clusters(flat_ground_truth_dict)
    precision, recall = precision_recall_microservice(pred_clusters, gold_clusters)
    metric_used = "Precision/Recall"

print("Hybrid Precision/Recall Evaluation")
print("----------------------------------")
print(f"Metric Used              : {metric_used}")
print(f"Precision                : {precision:.4f}")
print(f"Recall                   : {recall:.4f}")
print(f"# Predicted Partitions   : {num_predicted}")
print(f"# Ground Truth Partitions: {num_ground_truth}")
